# Final Year Project

## 22067567 Amogh Man Bajracharya

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import joblib
import json

In [2]:
#Reading both dataset and combining
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
combined_data = pd.concat([train, test], ignore_index=True)

C:\Users\legion\AppData\Local\Temp\ipykernel_21304\776894560.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train=pd.read_csv('train.csv')


In [3]:
print(combined_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        150000 non-null  object 
 1   Customer_ID               150000 non-null  object 
 2   Month                     150000 non-null  object 
 3   Name                      135000 non-null  object 
 4   Age                       150000 non-null  object 
 5   SSN                       150000 non-null  object 
 6   Occupation                150000 non-null  object 
 7   Annual_Income             150000 non-null  object 
 8   Monthly_Inhand_Salary     127500 non-null  float64
 9   Num_Bank_Accounts         150000 non-null  int64  
 10  Num_Credit_Card           150000 non-null  int64  
 11  Interest_Rate             150000 non-null  int64  
 12  Num_of_Loan               150000 non-null  object 
 13  Type_of_Loan              132888 non-null  o

In [4]:
print(combined_data.head())

       ID Customer_ID     Month           Name   Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265  Scientist   

  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0      19114.12            1824.843333                  3  ...           _   
1      19114.12                    NaN                  3  ...        Good   
2      19114.12                    NaN                  3  ...        Good   
3      19114.12                    NaN                  3  ...        Good   
4      19114.12            1824.843333                  3  ...        Good   

   Outstanding_Debt Credit_Utilization_Ratio     C

In [5]:
print(f"Dataset has {combined_data.shape[0]} rows and {combined_data.shape[1]} columns.")

Dataset has 150000 rows and 28 columns.


In [6]:
combined_data.drop_duplicates(inplace=True)

In [7]:
def parse_credit_age(age_str):
    if pd.isna(age_str):
        return None
    parts = age_str.split(' ')
    years = int(parts[0])
    months = int(parts[3]) if len(parts) > 3 else 0
    return years * 12 + months

In [8]:
combined_data['Credit_History_Age_Months'] = combined_data['Credit_History_Age'].apply(parse_credit_age)
combined_data['Credit_History_Age_Months'].fillna(combined_data['Credit_History_Age_Months'].median(), inplace=True)

In [9]:
combined_data['Num_of_Delayed_Payment'].fillna(0, inplace=True)

In [10]:
for column in combined_data.select_dtypes(include=['float64', 'int64']).columns:
    combined_data[column].fillna(combined_data[column].median(), inplace=True)

In [11]:
def convert_to_float(x):
    try:
        return float(x)
    except:
        return None

In [12]:
# Calculate the Credit Mix Number for each row
def calculate_credit_mix(row):
    credit_types = set()
    if row['Num_Credit_Card'] > 0:
        credit_types.add('Credit Card')
    if pd.notna(row['Type_of_Loan']):
        loans = row['Type_of_Loan'].split(',')
        for loan in loans:
            credit_types.add(loan.strip())
    return len(credit_types)

combined_data['Credit_Mix_Number'] = combined_data.apply(calculate_credit_mix, axis=1)

# Find the maximum Credit Mix Number for each individual
max_credit_mix_by_individual = combined_data.groupby('Name')['Credit_Mix_Number'].max()

# Map this max value back to each row for consistency across an individual's records
combined_data['Max_Credit_Mix_Number'] = combined_data['Name'].map(max_credit_mix_by_individual)

# Print some results to verify
print(combined_data[['Name', 'Credit_Mix_Number', 'Max_Credit_Mix_Number']].head())

            Name  Credit_Mix_Number  Max_Credit_Mix_Number
0  Aaron Maashoh                  5                    5.0
1  Aaron Maashoh                  5                    5.0
2  Aaron Maashoh                  5                    5.0
3  Aaron Maashoh                  5                    5.0
4  Aaron Maashoh                  5                    5.0


In [13]:
features_to_normalize = ['Credit_Utilization_Ratio', 'Credit_History_Age_Months', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries','Credit_Mix_Number']

In [14]:
for feature in features_to_normalize:
    # Convert to string, remove non-numeric characters if necessary, then convert to float
    combined_data[feature] = combined_data[feature].astype(str).str.extract('(\d+.\d+|\d+)')[0].apply(convert_to_float)

In [15]:
for feature in features_to_normalize:
    combined_data[feature].fillna(combined_data[feature].median(), inplace=True)

In [16]:
min_max_values = {}
for feature in features_to_normalize:
    max_val = combined_data[feature].max()
    min_val = combined_data[feature].min()
    combined_data[feature + '_norm'] = (combined_data[feature] - min_val) / (max_val - min_val)
    min_max_values[feature] = {'min': min_val, 'max': max_val}

In [17]:
print(combined_data[features_to_normalize + [f"{f}_norm" for f in features_to_normalize]].head())

   Credit_Utilization_Ratio  Credit_History_Age_Months  \
0                 26.822620                      265.0   
1                 31.944960                      222.0   
2                 28.609352                      267.0   
3                 31.377862                      268.0   
4                 24.797347                      269.0   

   Num_of_Delayed_Payment  Num_Credit_Inquiries  Credit_Mix_Number  \
0                     7.0                   4.0                5.0   
1                     0.0                   4.0                5.0   
2                     7.0                   4.0                5.0   
3                     4.0                   4.0                5.0   
4                     0.0                   4.0                5.0   

   Credit_Utilization_Ratio_norm  Credit_History_Age_Months_norm  \
0                       0.227421                        0.648649   
1                       0.398165                        0.542998   
2                       0.

In [18]:
combined_data['FICO_score_estimation'] = (
    0.35 * (1 - combined_data['Num_of_Delayed_Payment_norm']) +  # Less delayed = better
    0.30 * (1 - combined_data['Credit_Utilization_Ratio_norm']) +  # Less utilization = better
    0.15 * combined_data['Credit_History_Age_Months_norm'] +  # More age = better
    0.10 * (1 - combined_data['Num_Credit_Inquiries_norm']) +  # Less inquiries = better
    0.10 * combined_data['Credit_Mix_Number_norm']  # Good mix = better
) * 550 + 300

In [19]:
print(combined_data[['FICO_score_estimation']].head())

   FICO_score_estimation
0             755.598073
1             719.015304
2             746.176450
3             731.283628
4             767.854203


In [20]:
print(combined_data[['FICO_score_estimation']])

        FICO_score_estimation
0                  755.598073
1                  719.015304
2                  746.176450
3                  731.283628
4                  767.854203
...                       ...
149995             707.656118
149996             763.612314
149997             713.546106
149998             701.157911
149999             729.076580

[150000 rows x 1 columns]


In [21]:
max_score_index = combined_data['FICO_score_estimation'].idxmax()

In [22]:
highest_score_row = combined_data.loc[max_score_index]
print(highest_score_row)

ID                                                                          0x1f7cb
Customer_ID                                                              CUS_0xb518
Month                                                                      February
Name                                                                      Nobuhirot
Age                                                                              49
SSN                                                                     067-00-6687
Occupation                                                               Accountant
Annual_Income                                                             127855.83
Monthly_Inhand_Salary                                                    10511.6525
Num_Bank_Accounts                                                                 5
Num_Credit_Card                                                                   7
Interest_Rate                                                               

In [23]:
columns_to_save = [
    "Num_of_Delayed_Payment_norm",
    "Credit_Utilization_Ratio_norm",
    "Credit_History_Age_Months_norm",
    "Num_Credit_Inquiries_norm",
    'Credit_Mix_Number_norm',
    "FICO_score_estimation"
]

# Save the dataset to a CSV file
combined_data[columns_to_save].to_csv("processed_combined_data.csv", index=False)

print("CSV file saved successfully!")

CSV file saved successfully!


In [24]:
processed_data = pd.read_csv("processed_combined_data.csv")


In [25]:
print(processed_data.head())

   Num_of_Delayed_Payment_norm  Credit_Utilization_Ratio_norm  \
0                     0.001591                       0.227421   
1                     0.000000                       0.398165   
2                     0.001591                       0.286978   
3                     0.000909                       0.379262   
4                     0.000000                       0.159912   

   Credit_History_Age_Months_norm  Num_Credit_Inquiries_norm  \
0                        0.648649                    0.00154   
1                        0.542998                    0.00154   
2                        0.653563                    0.00154   
3                        0.656020                    0.00154   
4                        0.658477                    0.00154   

   Credit_Mix_Number_norm  FICO_score_estimation  
0                     0.5             755.598073  
1                     0.5             719.015304  
2                     0.5             746.176450  
3                   

In [26]:
features = [
    "Num_of_Delayed_Payment_norm",
    "Credit_Utilization_Ratio_norm",
    "Credit_History_Age_Months_norm",
    "Num_Credit_Inquiries_norm",
    'Credit_Mix_Number_norm',
]

In [27]:
target = "FICO_score_estimation"

In [28]:
if all(feature in processed_data.columns for feature in features) and target in processed_data.columns:
    print("✅ All required columns are present.")
else:
    missing_features = [feature for feature in features if feature not in processed_data.columns]
    print(f"❌ Missing features: {missing_features}")

✅ All required columns are present.


In [29]:
# Extract features (X) and target (y)
X = processed_data[features]
y = processed_data[target]

# Split into training (80%) and testing (20%) datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]} rows")
print(f"Testing set size: {X_test.shape[0]} rows")

Training set size: 120000 rows
Testing set size: 30000 rows


In [30]:
# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

print("✅ Model training complete.")

✅ Model training complete.


In [31]:
# Predict on test set
y_pred = model.predict(X_test)

# Predict on the entire dataset
processed_data["FICO_score_prediction"] = model.predict(X)

print("✅ Predictions generated.")


✅ Predictions generated.


In [32]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print results
print(f"📊 Model Performance Metrics:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.4f}")


📊 Model Performance Metrics:
Mean Squared Error (MSE): 0.00
Root Mean Squared Error (RMSE): 0.00
R-squared (R²): 1.0000


In [33]:
import joblib

# Save the trained model
joblib.dump(model, "credit_score_model.pkl")

print("✅ Model saved as credit_score_model.pkl")



✅ Model saved as credit_score_model.pkl


In [34]:
import joblib

# Load the trained model
model = joblib.load("credit_score_model.pkl")

# Example input for prediction
sample_input = [[1.2, 0, 5, 10,11]]
prediction = model.predict(sample_input)

print("Predicted Credit Score:", prediction)


Predicted Credit Score: [949.]


G:\fourth sem islington\anaconda\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [100]:
import joblib
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Load trained model
model = joblib.load("credit_score_model.pkl")

# Define input shape (change 5 to your number of features)
initial_type = [("float_input", FloatTensorType([None, 5]))]

# Convert model to ONNX format
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save ONNX model
with open("credit_score_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

print("ONNX model saved as credit_score_model.onnx")


ONNX model saved as credit_score_model.onnx


In [35]:
import os
print(os.getcwd())


C:\Users\legion\Final Year Project


In [36]:
import sys
print(sys.version)


3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [37]:
import os
print(os.getcwd())


C:\Users\legion\Final Year Project


In [38]:
import onnx

model = onnx.load("credit_score_model_v9.onnx")
print(f"ONNX IR Version: {model.ir_version}")


ONNX IR Version: 9


In [39]:
import onnx

# Load the existing ONNX model
model = onnx.load("credit_score_model.onnx")

# Change IR version to 9
model.ir_version = 9

# Save the converted model
onnx.save(model, "credit_score_model_v9.onnx")

print("Model successfully converted to IR version 9!")


Model successfully converted to IR version 9!


In [40]:
pip install onnx tf2onnx


Note: you may need to restart the kernel to use updated packages.


In [41]:
import onnx

# Load the original ONNX model
model_path = "credit_score_model.onnx"
model = onnx.load(model_path)

# Convert opset version to 19
converted_model = onnx.version_converter.convert_version(model, 19)

# Save the new model
onnx.save(converted_model, "credit_score_model_v19.onnx")

print("Model successfully converted to opset 19!")


Model successfully converted to opset 19!


In [42]:
import onnx
import onnxruntime as ort

# Load the original ONNX model
model_path = "credit_score_model.onnx"
model = onnx.load(model_path)

# Convert to Opset 19
target_opset = 19
converted_model = onnx.version_converter.convert_version(model, target_opset)

# Set IR version to 9
converted_model.ir_version = 9

# Save the new model
onnx.save(converted_model, "credit_score_model_fixed.onnx")

# Verify conversion
print("Model successfully converted!")
print(f"New IR Version: {converted_model.ir_version}")
for opset in converted_model.opset_import:
    print(f"New Opset Version: {opset.version}")
# Print the model's graph
print(model.graph)

Model successfully converted!
New IR Version: 9
New Opset Version: 1
New Opset Version: 19
node {
  input: "float_input"
  output: "variable"
  name: "LinearRegressor"
  op_type: "LinearRegressor"
  attribute {
    name: "coefficients"
    floats: 192.5
    floats: 165.0
    floats: 82.5
    floats: 55.0
    floats: 55.0
    type: FLOATS
  }
  attribute {
    name: "intercepts"
    floats: 300.0
    type: FLOATS
  }
  domain: "ai.onnx.ml"
}
name: "f994eccd07734b178a80b7ee46c97454"
input {
  name: "float_input"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
        }
        dim {
          dim_value: 5
        }
      }
    }
  }
}
output {
  name: "variable"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
        }
        dim {
          dim_value: 1
        }
      }
    }
  }
}



In [43]:
import onnx

# Load the model
model = onnx.load("credit_score_model_fixed.onnx")

# Print input names
for input in model.graph.input:
    print(f"Input name: {input.name}")


Input name: float_input


In [46]:
import onnx

model = onnx.load("credit_score_model_fixed.onnx")
for input in model.graph.input:
    print(f"Input Name: {input.name}, Shape: {input.type.tensor_type.shape.dim}")

Input Name: float_input, Shape: [, dim_value: 5
]


In [45]:

# Path to save the JSON file
json_file_path = 'min_max_values.json'

# Write the dictionary to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(min_max_values, json_file, indent=4)